In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"christopherkeibel","key":"3cfad0619d88b713dbe51f5399142891d9"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# downloading dataset from kaggle
!kaggle competitions download -c dogs-vs-cats

 98% 792M/812M [00:04<00:00, 217MB/s]
100% 812M/812M [00:04<00:00, 202MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [10]:
import os, shutil, pathlib

o_path = pathlib.Path("train")
new_path = pathlib.Path("dogs_vs_cats_small")

def make_subsets(name, start, end):
    for category in ("dog", "cat"):
        dir = new_path / name / category
        os.makedirs(dir)

        fnames = [f"{category}.{i}.jpg" for i in range(start, end)]
        for fname in fnames:
            shutil.copyfile(src=o_path / fname, dst= dir / fname)

In [11]:
make_subsets("train", 0, 1_000)
make_subsets("validation", 1_000, 1_500)
make_subsets("test", 1_500, 2_500)

In [13]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
    new_path / "train",
    image_size=(180, 180),
    batch_size=32
)

val_data = image_dataset_from_directory(
    new_path / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_data = image_dataset_from_directory(
    new_path / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


# Build the model

In [16]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, 
    Conv2D, 
    SeparableConv2D, 
    Activation, 
    Dense, 
    BatchNormalization, 
    MaxPooling2D, 
    RandomRotation, 
    RandomZoom, 
    RandomFlip, 
    Rescaling, 
    add,
    GlobalAveragePooling2D,
    Dropout
)

data_aug = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.2)
])

inputs = Input(shape=(180, 180, 3))
x = data_aug(inputs)
x = Rescaling(1./255)(x)
x = Conv2D(filters=32, kernel_size=5, use_bias=False)(x)

for size in [32, 64, 128, 256, 512]:
    residual = x

    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = SeparableConv2D(size, 3, padding="same", use_bias=False)(x)

    x = MaxPooling2D(3, strides=2, padding="same")(x)

    residual = Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)
    x = add([x, residual])

x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [ ]:
res = model.fit(train_data,
                epochs=100,
                validation_data=val_data)

Epoch 1/100
63/63 [==============================] - 45s 463ms/step - loss: 0.6930 - accuracy: 0.5650 - val_loss: 0.7025 - val_accuracy: 0.5000
Epoch 2/100
63/63 [==============================] - 28s 438ms/step - loss: 0.6572 - accuracy: 0.6055 - val_loss: 0.6924 - val_accuracy: 0.5350
Epoch 3/100
63/63 [==============================] - 27s 427ms/step - loss: 0.6363 - accuracy: 0.6440 - val_loss: 0.7084 - val_accuracy: 0.5000
Epoch 4/100
63/63 [==============================] - 28s 437ms/step - loss: 0.6200 - accuracy: 0.6580 - val_loss: 0.7078 - val_accuracy: 0.5000
Epoch 5/100
63/63 [==============================] - 28s 434ms/step - loss: 0.5980 - accuracy: 0.6810 - val_loss: 0.7856 - val_accuracy: 0.5000
Epoch 6/100
63/63 [==============================] - 28s 438ms/step - loss: 0.5749 - accuracy: 0.7070 - val_loss: 0.7867 - val_accuracy: 0.5000
Epoch 7/100
63/63 [==============================] - 28s 441ms/step - loss: 0.5709 - accuracy: 0.7070 - val_loss: 1.0092 - val_accuracy:

In [ ]:
loss = res.history["loss"]
acc = res.history["accuracy"]
val_loss = res.history["val_loss"]
vall_acc = res.history["val_accuracy"]

x_range = range(0, len(loss) + 1)

import matplotlib.pyplot as plt

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 8))
ax1.plot(x_range, acc, "b", label="Train acc")
ax1.plot(x_range, val_acc, "r", label="Val acc")
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Accuracy")
ax1.set_title("Accuracy")
ax1.legend()
ax2.plot(x_range, loss, "b", label="Train loss")
ax2.plot(x_range, val_loss, "r", label="Val loss")
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.legend()
ax2.set_title("Loss")
plt.show()